In [9]:
!brew install libomp

'brew' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!export DYLD_LIBRARY_PATH=$(brew --prefix libomp)/lib:$DYLD_LIBRARY_PATH

'export' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!echo 'export DYLD_LIBRARY_PATH=$(brew --prefix libomp)/lib:$DYLD_LIBRARY_PATH' >> ~/.zshrc
!source ~/.zshrc

The system cannot find the path specified.
'source' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import os
os.chdir(os.getcwd().replace('/notebooks', ''))

# Đọc dữ liệu đã tiền xử lý
train_filepath = '../data/preprocessed/train_preprocessed.csv'
test_filepath = '../data/preprocessed/test_preprocessed.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)

# Tách features và target
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

# Chia dữ liệu train - validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo model XGBoost
model = xgb.XGBRegressor(
    n_estimators=500, 
    learning_rate=0.05, 
    max_depth=6, 
    subsample=0.8, 
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

# Huấn luyện mô hình
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

# Dự đoán trên tập test
test_predictions = model.predict(test_data)

with open('../data/preprocessed/scaling_params.txt', 'r') as f:
    lines = f.readlines()
    mean = float(lines[0].split()[1])
    std = float(lines[1].split()[1])

# Đảo ngược chuẩn hóa
test_predictions = test_predictions * std + mean

# Lưu kết quả
output_dir = './data/output'
os.makedirs(output_dir, exist_ok=True)
output_filepath = os.path.join(output_dir, 'predictions.csv')
test_ids = pd.read_csv(test_filepath)['Id']
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions})
submission.to_csv(output_filepath, index=False)

print(f"Predictions saved to {output_filepath}")

[0]	validation_0-rmse:1.06111
[1]	validation_0-rmse:1.02462
[2]	validation_0-rmse:0.98923
[3]	validation_0-rmse:0.95575
[4]	validation_0-rmse:0.92493
[5]	validation_0-rmse:0.89651
[6]	validation_0-rmse:0.86538
[7]	validation_0-rmse:0.83634
[8]	validation_0-rmse:0.80984
[9]	validation_0-rmse:0.78419
[10]	validation_0-rmse:0.75754
[11]	validation_0-rmse:0.73968
[12]	validation_0-rmse:0.71567
[13]	validation_0-rmse:0.69504
[14]	validation_0-rmse:0.67280
[15]	validation_0-rmse:0.65219
[16]	validation_0-rmse:0.63357
[17]	validation_0-rmse:0.61639
[18]	validation_0-rmse:0.60141
[19]	validation_0-rmse:0.58618
[20]	validation_0-rmse:0.56909
[21]	validation_0-rmse:0.55554
[22]	validation_0-rmse:0.54246
[23]	validation_0-rmse:0.53132
[24]	validation_0-rmse:0.52069
[25]	validation_0-rmse:0.50965
[26]	validation_0-rmse:0.49867
[27]	validation_0-rmse:0.48896
[28]	validation_0-rmse:0.47941
[29]	validation_0-rmse:0.47039
[30]	validation_0-rmse:0.46240
[31]	validation_0-rmse:0.45416
[32]	validation_0-